# SnapShot preprocessing:
In this notebook we get the MATSim snapShot output as the input. MATSim gives us the user location in constant intervals. It does not report anything when the user is not traveling. So to make this data as much as possible closer to the actual periodic location update, we have to fill in the empty records. also we initially get the snapshots every 30 seconds. For other polling intervals we only reduce the accuracy. Also for the spatial aggregation to the level os TAZ zones, we use arcGIS to extract the associated TAZ of each location record from the snapShot file. The output of arcGIS is the input here.

#### adding required packages

In [117]:
import pandas as pd
import time
from math import floor
import numpy as np
import pickle
import requests
import concurrent.futures

#### specifying the saving location 

In [3]:
savingLoc = "Y:/ZahraEftekhar/phase4/"

#### preparing the output of arcGIS for completion

The output of GIS misses the locations outside of Amsterdam. Therefore, we complete the data by considering their TAZ code `0`.

In [41]:
precompletion = pd.read_csv('{a}GISoutput_PreCompletion.CSV'.format(a=savingLoc),usecols=['mzr_id', 'VEHICLE','TIME','EASTING','NORTHING'])
precompletion = precompletion.sort_values(by=["VEHICLE","TIME"])
precompletion = precompletion.reset_index(drop=True)
with open('{a}snapShot_allowedUsers.pickle'.format(a=savingLoc),'rb') as handle:
    MATSimOutput = pickle.load(handle)
MATSimOutput=MATSimOutput.reset_index(drop=False)
print(len(MATSimOutput)-len(precompletion)," records are missing that we refill them in our snapShot. ")


7481  records are missing that we refill them in our snapShot. 


In [42]:
snapData = pd.merge(precompletion, MATSimOutput, how='right', on=['VEHICLE','TIME'])
(snapData.mzr_id[snapData.mzr_id.isna()]) = 0
snapData = snapData.loc[:,['VEHICLE','TIME','EASTING_y','NORTHING_y','mzr_id']]
snapData.columns = ['VEHICLE', 'TIME', 'EASTING', 'NORTHING', 'mzr_id']
snapData = snapData.sort_values(by = ['VEHICLE', 'TIME'])
with open('{a}finalInputPython.pickle'.format(a=savingLoc),'wb') as handle:
    pickle.dump(snapData, handle, protocol=pickle.HIGHEST_PROTOCOL)

c:\programdata\anaconda3\envs\extractingodfromxml\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Now, it is time to generate the complete snapShot data for every 30 seconds which represents the base data set even for generating other snapshots with different polling intervals (we resample from this data based on the specified polling interval).

In [118]:
with open('{a}finalInputPython.pickle'.format(a=savingLoc),'rb') as handle:
    snapData = pickle.load(handle)
snapData.reset_index(drop=True,inplace=True)
snapData.head()

,VEHICLE,TIME,EASTING,NORTHING,mzr_id
0,1,23400,632364.770972,5.816900e+06,7065.0
1,1,23430,632279.680941,5.816846e+06,5329.0
2,1,23460,632234.315601,5.816431e+06,5329.0
3,1,23490,632200.291596,5.816119e+06,5329.0
4,1,23520,632209.756236,5.815776e+06,5329.0


In [114]:
userGroups = (snapData.groupby(["VEHICLE"]))
kkk = {}
# for ID in 
kk = userGroups.get_group(1)
kk.TIME = pd.to_timedelta(kk.TIME, unit="s")
kk.set_index(["TIME"],inplace=True)
# print(kk.tail())
kk.loc[kk.index[0]+pd.to_timedelta('24:00:00')]=kk.iloc[0,:]
kk.sort_index(inplace=True)
kk = kk.resample('30S').fillna("pad")
kk.drop([kk.index[0]+pd.to_timedelta('24:00:00')],axis=0,inplace=True)
# kk.reset_index(drop=False, inplace=True)
kkk[1] = kk
kk = userGroups.get_group(30)
kk.TIME = pd.to_timedelta(kk.TIME, unit="s")
kk.set_index(["TIME"],inplace=True)
# print(kk.tail())
kk.loc[kk.index[0]+pd.to_timedelta('24:00:00')]=kk.iloc[0,:]
kk.sort_index(inplace=True)
kk = kk.resample('30S').fillna("pad")
kk.drop([kk.index[0]+pd.to_timedelta('24:00:00')],axis=0,inplace=True)
# kk.reset_index(drop=False, inplace=True)
kkk[30] = kk
# kkk = pd.DataFrame(kkk)
print(kkk)

{1:                  VEHICLE        EASTING      NORTHING  mzr_id
TIME                                                         
0 days 06:30:00      1.0  632364.770972  5.816900e+06  7065.0
0 days 06:30:30      1.0  632279.680941  5.816846e+06  5329.0
0 days 06:31:00      1.0  632234.315601  5.816431e+06  5329.0
0 days 06:31:30      1.0  632200.291596  5.816119e+06  5329.0
0 days 06:32:00      1.0  632209.756236  5.815776e+06  5329.0
...                  ...            ...           ...     ...
1 days 06:27:30      1.0  632318.660822  5.816858e+06  5329.0
1 days 06:28:00      1.0  632318.660822  5.816858e+06  5329.0
1 days 06:28:30      1.0  632318.660822  5.816858e+06  5329.0
1 days 06:29:00      1.0  632318.660822  5.816858e+06  5329.0
1 days 06:29:30      1.0  632318.660822  5.816858e+06  5329.0

[2880 rows x 4 columns], 30:                  VEHICLE        EASTING      NORTHING  mzr_id
TIME                                                         
0 days 06:19:30     30.0  640266.661

c:\programdata\anaconda3\envs\extractingodfromxml\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
c:\programdata\anaconda3\envs\extractingodfromxml\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [110]:
#we time the process
startTime = time.time()

userGroups = (snapData.groupby(["VEHICLE"]))
concatData = {}
for ID in userGroups.groups.keys():
    records=userGroups.get_group(ID)
    records.TIME = pd.to_timedelta(records.TIME, unit="s")
    records.set_index(["TIME"],inplace=True)
    # print(kk.tail())
    records.loc[records.index[0]+pd.to_timedelta('24:00:00')]=records.iloc[0,:]
    records.sort_index(inplace=True)
    records = records.resample('30S').fillna("pad")
    records.drop([records.index[0]+pd.to_timedelta('24:00:00')],axis=0,inplace=True)
    # kk.reset_index(drop=False, inplace=True)
    concatData[ID] = kk
with open("completePLUdata_30sec_dict.pickle".format(a=savingLoc),"wb") as handle:
          pickle.dump(concatData,handle, protocol=pickle.HIGHEST_PROTOCOL )
snapDataNew = pd.DataFrame()
for ID in concatData.keys():
    snapDataNew = snapDataNew.append(concatData[ID])
with open("completePLUdata_30sec.pickle".format(a=savingLoc),"wb") as handle:
          pickle.dump(snapDataNew,handle, protocol=pickle.HIGHEST_PROTOCOL )
print((time.time() - startTime)//60,'minutes')

c:\programdata\anaconda3\envs\extractingodfromxml\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
c:\programdata\anaconda3\envs\extractingodfromxml\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


KeyboardInterrupt: 

In [125]:
#we time the process
startTime = time.time()
concatData = pd.DataFrame()
userGroups = (snapData.groupby(["VEHICLE"]))
# IDs = [1,6,30,100007,100009,10001,100011,100014,100048,100054]
def completeRecords(records):#records is a dataframe
    records.TIME = pd.to_timedelta(records.TIME, unit="s")
    records.set_index(["TIME"],inplace=True)
    records.loc[records.index[0]+pd.to_timedelta('24:00:00')]=records.iloc[0,:]
    records.sort_index(inplace=True)
    records = records.resample('30S').fillna("pad")
    records.drop([records.index[0]+pd.to_timedelta('24:00:00')],axis=0,inplace=True)
    return records
futures = []
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    for ID in userGroups.groups.keys():#userGroups.groups.keys(),IDs
        futures.append(executor.submit(completeRecords, records=userGroups.get_group(ID)))
    for future in concurrent.futures.as_completed(futures):
        concatData = concatData.append(future.result())
with open("completePLUdata_30sec.pickle".format(a=savingLoc),"wb") as handle:
          pickle.dump(snapDataNew,handle, protocol=pickle.HIGHEST_PROTOCOL )
print((time.time() - startTime)//60,'minutes')


0.0 minutes


In [126]:
concatData

,VEHICLE,EASTING,NORTHING,mzr_id
TIME,,,,
0 days 06:30:00,1.0,632364.770972,5.816900e+06,7065.0
0 days 06:30:30,1.0,632279.680941,5.816846e+06,5329.0
0 days 06:31:00,1.0,632234.315601,5.816431e+06,5329.0
0 days 06:31:30,1.0,632200.291596,5.816119e+06,5329.0
0 days 06:32:00,1.0,632209.756236,5.815776e+06,5329.0
...,...,...,...,...
1 days 05:17:00,100054.0,629786.783744,5.802847e+06,7514.0
1 days 05:17:30,100054.0,629786.783744,5.802847e+06,7514.0
1 days 05:18:00,100054.0,629786.783744,5.802847e+06,7514.0
